# AWS ECS Task Definition That Queries The Credential Endpoint

Detects when an Elastic Container Service (ECS) Task Definition includes a command to query the credential endpoint.
This can indicate a potential adversary adding a backdoor to establish persistence or escalate privileges.


References:
* https://github.com/RhinoSecurityLabs/pacu/blob/866376cd711666c775bbfcde0524c817f2c5b181/pacu/modules/ecs__backdoor_task_def/main.py
* https://docs.aws.amazon.com/AmazonECS/latest/APIReference/API_RegisterTaskDefinition.html
* https://docs.aws.amazon.com/AmazonECS/latest/developerguide/task-iam-roles.html

False Positives:
* Task Definition being modified to request credentials from the Task Metadata Service for valid reasons

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource="ecs.amazonaws.com"
eventName=(DescribeTaskDefinition RegisterTaskDefinition RunTask)
requestParameters.containerDefinitions.command:'*AWS_CONTAINER_CREDENTIALS_RELATIVE_URI*'
| stats
min(@scnr.datetime) as firstTime,
max(@scnr.datetime) as lastTime,
count() as eventCount
by
userIdentity.arn,
eventSource,
eventName,
awsRegion

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName